<a href="https://colab.research.google.com/github/lhfp98/git-bootcamp/blob/main/gibson_OCR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:

import re
import pdfplumber

import re
import pdfplumber

def extract_date_from_pdf(pdf_path):
    try:
        # Open the PDF file
        with pdfplumber.open(pdf_path) as pdf:
            # Get the first page
            first_page = pdf.pages[0]
            # Extract text from the first page
            text = first_page.extract_text()

            # Normalize the text (strip extra spaces or formatting issues)
            normalized_text = " ".join(text.split())

            # Updated regex pattern to capture ordinal suffixes and proper month names
            date_pattern = r'\b\d{1,2}(?:st|nd|rd|th)?\s+(January|February|March|April|May|June|July|August|September|October|November|December)\s+\d{4}\b'
            date_matches = re.findall(date_pattern, normalized_text, re.IGNORECASE)

            if date_matches:
                return date_matches[0]
            else:
                return "Date not found"
    except Exception as e:
        return f"Error processing file {pdf_path}: {e}"

# Path to the uploaded file
pdf_path = "/gibson/2020 GIBSON GAS - WEEK (26).pdf"

# Call the function
date = extract_date_from_pdf(pdf_path)
print(f"Extracted Date: {date}")




Extracted Date: June


TESTE MED BILDE PROSSESERING


In [25]:
import re
import pytesseract
from PIL import Image
import pdfplumber

# Path to the PDF file
pdf_path = "/gibson/2021 GIBSON GAS - WEEK (26).pdf"

def extract_date_from_pdf(pdf_path):
    try:
        # Open the PDF file
        with pdfplumber.open(pdf_path) as pdf:
            # Get the first page
            first_page = pdf.pages[0]
            # Extract text from the first page
            text = first_page.extract_text()

            # Regex pattern for the date
            date_pattern = r'\b\d{1,2}(?:st|nd|rd|th)?\s+(January|February|March|April|May|June|July|August|September|October|November|December)\s+\d{4}\b'
            date_matches = re.findall(date_pattern, text, re.IGNORECASE)

            if date_matches:
                return date_matches[0]

            # If date is not found, fallback to OCR
            print("Date not found using regex. Falling back to OCR...")
            return extract_date_from_top_right_corner(first_page)
    except Exception as e:
        return f"Error processing PDF: {e}"

def extract_date_from_top_right_corner(page):
    try:
        # Convert the first page to an image
        pdf_image = page.to_image(resolution=300)  # Increase resolution for better OCR
        original_image = pdf_image.original

        # Debug: Save the full-page image to check alignment
        original_image.save("/gibson/full_page_image.png")

        # Crop the top-right corner of the image (adjust as necessary)
        width, height = original_image.size
        crop_box = (int(width * 0.7), 0, width, int(height * 0.15))  # Focus on the top-right corner
        cropped_image = original_image.crop(crop_box)

        # Debug: Save the cropped image for verification
        cropped_image.save("/gibson/cropped_top_right.png")

        # Convert the cropped image to grayscale
        grayscale_image = cropped_image.convert('L')

        # Debug: Save the grayscale image for verification
        grayscale_image.save("/gibson/cropped_grayscale.png")

        # Perform OCR to extract text
        text = pytesseract.image_to_string(grayscale_image)

        # Debug: Print the OCR result for verification
        print(f"OCR Result: {text}")

        # Regex pattern for the date
        date_pattern = r'\b\d{1,2}(?:st|nd|rd|th)?\s+(January|February|March|April|May|June|July|August|September|October|November|December)\s+\d{4}\b'
        match = re.search(date_pattern, text, re.IGNORECASE)

        if match:
            return match.group(0)
        else:
            return "Date not found in OCR"
    except Exception as e:
        return f"Error processing image: {e}"

# Call the function
date = extract_date_from_pdf(pdf_path)
print(f"Extracted Date: {date}")


Date not found using regex. Falling back to OCR...
Extracted Date: Error processing image: /usr/bin/tesseract is not installed or it's not in your PATH. See README file for more information.


In [13]:
import os
import re
import pdfplumber

# Path to the folder containing PDF files
folder_path = "/gibson/2021 GIBSON GAS - WEEK (26).pdf"

def extract_date_from_pdf(pdf_path):
    try:
        # Open the PDF file
        with pdfplumber.open(pdf_path) as pdf:
            # Get the first page
            first_page = pdf.pages[0]
            # Extract text from the first page
            text = first_page.extract_text()

            # Updated regex pattern to capture ordinal suffixes and proper month names
            date_pattern = r'\b\d{1,2}(?:st|nd|rd|th)?\s+(January|February|March|April|May|June|July|August|September|October|November|December)\s+\d{4}\b'
            date_matches = re.findall(date_pattern, text, re.IGNORECASE)

            if date_matches:
                return date_matches[0]
            else:
                return "Date not found"
    except Exception as e:
        return f"Error processing file {pdf_path}: {e}"

def extract_dates_from_folder(folder_path):
    dates = {}
    # List all files in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith(".pdf"):
            pdf_path = os.path.join(folder_path, filename)
            date = extract_date_from_pdf(pdf_path)
            dates[filename] = date
    return dates

# Extract dates from all PDFs in the folder
dates = extract_dates_from_folder(folder_path)

# Print the extracted dates
for filename, date in dates.items():
    print(f"{filename}: {date}")


NotADirectoryError: [Errno 20] Not a directory: '/gibson/2021 GIBSON GAS - WEEK (26).pdf'

In [9]:
# Extract dates from all PDFs in the folder
dates = extract_dates_from_folder(folder_path)

# Filter and print files where the date was not found
print("\nFiles with 'Date not found':")
for filename, date in dates.items():
    if date == "Date not found":
        print(f"{filename}: {date}")



Files with 'Date not found':
2021 GIBSON GAS - WEEK (34).pdf: Date not found
2021 GIBSON GAS - WEEK (26).pdf: Date not found
2021 GIBSON GAS - WEEK (25).pdf: Date not found
